In [164]:
##### derived from https://github.com/bozhu/AES-Python
import copy

Sbox = (
    0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
    0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
    0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
    0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
    0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
    0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
    0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
    0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
    0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
    0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
    0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
    0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
    0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
    0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
    0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
    0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16,
)

InvSbox = (
    0x52, 0x09, 0x6A, 0xD5, 0x30, 0x36, 0xA5, 0x38, 0xBF, 0x40, 0xA3, 0x9E, 0x81, 0xF3, 0xD7, 0xFB,
    0x7C, 0xE3, 0x39, 0x82, 0x9B, 0x2F, 0xFF, 0x87, 0x34, 0x8E, 0x43, 0x44, 0xC4, 0xDE, 0xE9, 0xCB,
    0x54, 0x7B, 0x94, 0x32, 0xA6, 0xC2, 0x23, 0x3D, 0xEE, 0x4C, 0x95, 0x0B, 0x42, 0xFA, 0xC3, 0x4E,
    0x08, 0x2E, 0xA1, 0x66, 0x28, 0xD9, 0x24, 0xB2, 0x76, 0x5B, 0xA2, 0x49, 0x6D, 0x8B, 0xD1, 0x25,
    0x72, 0xF8, 0xF6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xD4, 0xA4, 0x5C, 0xCC, 0x5D, 0x65, 0xB6, 0x92,
    0x6C, 0x70, 0x48, 0x50, 0xFD, 0xED, 0xB9, 0xDA, 0x5E, 0x15, 0x46, 0x57, 0xA7, 0x8D, 0x9D, 0x84,
    0x90, 0xD8, 0xAB, 0x00, 0x8C, 0xBC, 0xD3, 0x0A, 0xF7, 0xE4, 0x58, 0x05, 0xB8, 0xB3, 0x45, 0x06,
    0xD0, 0x2C, 0x1E, 0x8F, 0xCA, 0x3F, 0x0F, 0x02, 0xC1, 0xAF, 0xBD, 0x03, 0x01, 0x13, 0x8A, 0x6B,
    0x3A, 0x91, 0x11, 0x41, 0x4F, 0x67, 0xDC, 0xEA, 0x97, 0xF2, 0xCF, 0xCE, 0xF0, 0xB4, 0xE6, 0x73,
    0x96, 0xAC, 0x74, 0x22, 0xE7, 0xAD, 0x35, 0x85, 0xE2, 0xF9, 0x37, 0xE8, 0x1C, 0x75, 0xDF, 0x6E,
    0x47, 0xF1, 0x1A, 0x71, 0x1D, 0x29, 0xC5, 0x89, 0x6F, 0xB7, 0x62, 0x0E, 0xAA, 0x18, 0xBE, 0x1B,
    0xFC, 0x56, 0x3E, 0x4B, 0xC6, 0xD2, 0x79, 0x20, 0x9A, 0xDB, 0xC0, 0xFE, 0x78, 0xCD, 0x5A, 0xF4,
    0x1F, 0xDD, 0xA8, 0x33, 0x88, 0x07, 0xC7, 0x31, 0xB1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xEC, 0x5F,
    0x60, 0x51, 0x7F, 0xA9, 0x19, 0xB5, 0x4A, 0x0D, 0x2D, 0xE5, 0x7A, 0x9F, 0x93, 0xC9, 0x9C, 0xEF,
    0xA0, 0xE0, 0x3B, 0x4D, 0xAE, 0x2A, 0xF5, 0xB0, 0xC8, 0xEB, 0xBB, 0x3C, 0x83, 0x53, 0x99, 0x61,
    0x17, 0x2B, 0x04, 0x7E, 0xBA, 0x77, 0xD6, 0x26, 0xE1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0C, 0x7D,
)

Rcon = (
    0x00, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40,
    0x80, 0x1B, 0x36, 0x6C, 0xD8, 0xAB, 0x4D, 0x9A,
    0x2F, 0x5E, 0xBC, 0x63, 0xC6, 0x97, 0x35, 0x6A,
    0xD4, 0xB3, 0x7D, 0xFA, 0xEF, 0xC5, 0x91, 0x39,
)


def text2matrix(text): ##
    matrix = []
    for i in range(16):
        byte = (text >> (8 * (15 - i))) & 0xFF
        if i % 4 == 0:
            matrix.append([byte])
        else:
            matrix[i // 4].append(byte)
            
    #print("{:32x}".format(text))
    #print([[hex(a) for a in m] for m in matrix])
    
    """
        A B C D E F G H I J K L M N O P

        A B C D
        E F G H
        I J K L
        M N O P
    """
    
    return matrix


def matrix2text(matrix): ##
    text = 0
    for i in range(4):
        for j in range(4):
            text |= (matrix[i][j] << (120 - 8 * (4 * i + j)))
    return text


class AES:
    def __init__(self, master_key, iv=None):
        self.change_key(master_key)
        self.iv = iv
    
    def matrix_xor_elementwise(self, s, k): ##
        for i in range(4):
            for j in range(4):
                s[i][j] ^= k[i][j]
                
                
    # shifts / movements only
    def matrix_shift_rows(self, s): ##
        s[0][1], s[1][1], s[2][1], s[3][1] = s[1][1], s[2][1], s[3][1], s[0][1]
        s[0][2], s[1][2], s[2][2], s[3][2] = s[2][2], s[3][2], s[0][2], s[1][2]
        s[0][3], s[1][3], s[2][3], s[3][3] = s[3][3], s[0][3], s[1][3], s[2][3]

    def matrix_unshift_rows(self, s): ##
        s[0][1], s[1][1], s[2][1], s[3][1] = s[3][1], s[0][1], s[1][1], s[2][1]
        s[0][2], s[1][2], s[2][2], s[3][2] = s[2][2], s[3][2], s[0][2], s[1][2]
        s[0][3], s[1][3], s[2][3], s[3][3] = s[1][3], s[2][3], s[3][3], s[0][3]
        
    def matrix_sbox_lookup(self, s): ##
        for i in range(4):
            for j in range(4):
                s[i][j] = Sbox[s[i][j]]
                
    def matrix_invsbox_lookup(self, s): ##
        for i in range(4):
            for j in range(4):
                s[i][j] = InvSbox[s[i][j]]
                
                
    def mix_columns(self, s):
        xtime = lambda a: (((a << 1) ^ 0x1B) & 0xFF) if (a & 0x80) else (a << 1)
        for i in range(4):
            t = s[i][0] ^ s[i][1] ^ s[i][2] ^ s[i][3]
            u = s[i][0]
            s[i][0] ^= t ^ xtime(s[i][0] ^ s[i][1])
            s[i][1] ^= t ^ xtime(s[i][1] ^ s[i][2])
            s[i][2] ^= t ^ xtime(s[i][2] ^ s[i][3])
            s[i][3] ^= t ^ xtime(s[i][3] ^ u)


    def unmix_columns(self, s):
        xtime = lambda a: (((a << 1) ^ 0x1B) & 0xFF) if (a & 0x80) else (a << 1)

        for i in range(4):
            u = xtime(xtime(s[i][0] ^ s[i][2]))
            v = xtime(xtime(s[i][1] ^ s[i][3]))
            s[i][0] ^= u
            s[i][1] ^= v
            s[i][2] ^= u
            s[i][3] ^= v

        self.mix_columns(s)
        
    def encrypt(self, plaintext):
        if self.iv is not None:
            self.plain_state = text2matrix(plaintext ^ self.iv)
        else:
            self.plain_state = text2matrix(plaintext)

        self.matrix_xor_elementwise(self.plain_state, self.round_keys[0])
            
        #print([hex(x) for x in self.plain_state[0]])

        for i in range(1, 15):
            ## CYCLE 1
            self.matrix_sbox_lookup(self.plain_state)
            self.matrix_shift_rows(self.plain_state)
            
            #print([hex(x) for x in self.plain_state[0]])
            
            ## CYCLE 2
            if i != 14: self.mix_columns(self.plain_state)
            self.matrix_xor_elementwise(self.plain_state, self.round_keys[i])
            
            #print([hex(x) for x in self.plain_state[0]])
            
        if self.iv is not None:
            self.iv = matrix2text(self.plain_state)
        
        return matrix2text(self.plain_state)

    def decrypt(self, ciphertext):
        self.cipher_state = text2matrix(ciphertext)

        #print(hex(self.cipher_state[3][3]))
        
        for i in range(14, 0, -1):
            ## CYCLE 1
            self.matrix_xor_elementwise(self.cipher_state, self.round_keys[i])
            if i != 14: self.unmix_columns(self.cipher_state)
                
            #print(hex(self.cipher_state[3][3]))
                
            ## CYCLE 2
            self.matrix_unshift_rows(self.cipher_state)
            self.matrix_invsbox_lookup(self.cipher_state)
            
            #print(hex(self.cipher_state[0][3]))
  
        self.matrix_xor_elementwise(self.cipher_state, self.round_keys[0])
        
        out = matrix2text(self.cipher_state)
        
        if self.iv is not None:
            out = out ^ self.iv
            self.iv = ciphertext

        return out


    def change_key(self, master_key):
        self.round_keys = [text2matrix(master_key >> 128), text2matrix(master_key & ((1 << 128) - 1))]
        
        last_key2 = self.round_keys[0]
        last_key = self.round_keys[1]

        #print([hex(x) for x in last_key[0]])
            
        for i in range(1, 14):
            if i % 2 == 1:
                key = []

                # row 0
                r0b0 = last_key2[0][0] ^ Sbox[last_key[3][1]] ^ Rcon[i // 2 + 1]
                r0b1 = last_key2[0][1] ^ Sbox[last_key[3][2]]
                r0b2 = last_key2[0][2] ^ Sbox[last_key[3][3]]
                r0b3 = last_key2[0][3] ^ Sbox[last_key[3][0]]
                key.append([r0b0, r0b1, r0b2, r0b3])

                # row 1
                r1b0 = last_key2[1][0] ^ r0b0
                r1b1 = last_key2[1][1] ^ r0b1
                r1b2 = last_key2[1][2] ^ r0b2
                r1b3 = last_key2[1][3] ^ r0b3
                key.append([r1b0, r1b1, r1b2, r1b3])

                # row 2
                r2b0 = last_key2[2][0] ^ r1b0
                r2b1 = last_key2[2][1] ^ r1b1
                r2b2 = last_key2[2][2] ^ r1b2
                r2b3 = last_key2[2][3] ^ r1b3
                key.append([r2b0, r2b1, r2b2, r2b3])

                # row 3
                r3b0 = last_key2[3][0] ^ r2b0
                r3b1 = last_key2[3][1] ^ r2b1
                r3b2 = last_key2[3][2] ^ r2b2
                r3b3 = last_key2[3][3] ^ r2b3
                key.append([r3b0, r3b1, r3b2, r3b3])
            else:
                key = []

                # row 0
                r0b0 = last_key2[0][0] ^ Sbox[last_key[3][0]] #^ Rcon[i]
                r0b1 = last_key2[0][1] ^ Sbox[last_key[3][1]]
                r0b2 = last_key2[0][2] ^ Sbox[last_key[3][2]]
                r0b3 = last_key2[0][3] ^ Sbox[last_key[3][3]]
                key.append([r0b0, r0b1, r0b2, r0b3])

                # row 1
                r1b0 = last_key2[1][0] ^ r0b0
                r1b1 = last_key2[1][1] ^ r0b1
                r1b2 = last_key2[1][2] ^ r0b2
                r1b3 = last_key2[1][3] ^ r0b3
                key.append([r1b0, r1b1, r1b2, r1b3])

                # row 2
                r2b0 = last_key2[2][0] ^ r1b0
                r2b1 = last_key2[2][1] ^ r1b1
                r2b2 = last_key2[2][2] ^ r1b2
                r2b3 = last_key2[2][3] ^ r1b3
                key.append([r2b0, r2b1, r2b2, r2b3])

                # row 3
                r3b0 = last_key2[3][0] ^ r2b0
                r3b1 = last_key2[3][1] ^ r2b1
                r3b2 = last_key2[3][2] ^ r2b2
                r3b3 = last_key2[3][3] ^ r2b3
                key.append([r3b0, r3b1, r3b2, r3b3])
        
            self.round_keys.append(key)
            last_key2 = last_key
            last_key = key

            #print([hex(x) for x in key[0]])
        
        

In [165]:
def bytes2matrix(text):
    """ Converts a 16-byte array into a 4x4 matrix.  """
    return [list(text[i:i+4]) for i in range(0, len(text), 4)]

def ek(master_key, rounds=14):
    """
    Expands and returns a list of key matrices for the given master_key.
    """
    key_columns = bytes2matrix(master_key)

    i = 1
    while len(key_columns) < (rounds + 1) * 4:
        word = list(key_columns[-1])
        orig = word[:]

        if len(key_columns) % 8 == 0:
            word.append(word.pop(0))
            word = [Sbox[(b)] for b in word]
            word[0] ^= Rcon[i]
            i += 1
            #print("screm")
        elif len(key_columns) % 8 == 4:
            word = [Sbox[(b)] for b in word]
            #print("screm2")

        word = xor_bytes(word, key_columns[-8])
        #print("w", list(orig), list(word))
        key_columns.append(word)

    return [key_columns[4*i : 4*(i+1)] for i in range(len(key_columns) // 4)]

def xor_bytes(a, b):
    """ Returns a new byte array with the elements xor'ed. """
    return bytes(i^j for i, j in zip(a, b))

In [166]:
#print(ek(key.encode("ascii")))

a = AES(packtext(key))
print()

print("\n".join(map(repr, a.round_keys)))

print()
rk = ek(key.encode("ascii"))
print()

rk = [[list(b) for b in a] for a in rk]

print()
#a.round_keys = rk
print("\n".join(map(repr, rk)))

hh = (a.encrypt(packtext(string1)))
d = unpacktext(a.decrypt(hh))

print(hex(hh), d)

assert hh == 0x2d1541c695f88a16f8bfb5dbe3a95022


[[97, 98, 99, 100], [49, 50, 51, 52], [65, 66, 67, 68], [33, 64, 35, 36]]
[[122, 121, 120, 119], [90, 89, 88, 87], [42, 49, 42, 50], [42, 51, 42, 52]]
[[163, 135, 123, 129], [146, 181, 72, 181], [211, 247, 11, 241], [242, 183, 40, 213]]
[[243, 208, 76, 116], [169, 137, 20, 35], [131, 184, 62, 17], [169, 139, 20, 37]]
[[156, 125, 68, 82], [14, 200, 12, 231], [221, 63, 7, 22], [47, 136, 47, 195]]
[[230, 20, 89, 90], [79, 157, 77, 121], [204, 37, 115, 104], [101, 174, 103, 77]]
[[124, 248, 167, 31], [114, 48, 171, 248], [175, 15, 172, 238], [128, 135, 131, 45]]
[[43, 3, 181, 130], [100, 158, 248, 251], [168, 187, 139, 147], [205, 21, 236, 222]]
[[45, 54, 186, 162], [95, 6, 17, 90], [240, 9, 189, 180], [112, 142, 62, 153]]
[[122, 26, 7, 108], [30, 132, 255, 151], [182, 63, 116, 4], [123, 42, 152, 218]]
[[216, 112, 237, 131], [135, 118, 252, 217], [119, 127, 65, 109], [7, 241, 127, 244]]
[[191, 187, 213, 211], [161, 63, 42, 68], [23, 0, 94, 64], [108, 42, 198, 154]]
[[29, 196, 85, 211], [1

In [146]:
# 2d1541c695f88a16f8bfb5dbe3a95022

def packtext(s):
    s = s
    o = 0
    while len(s) > 0:
        o = (o << 8) | ord(s[0])
        s = s[1:]
    return o

def unpacktext(s):
    o = ""
    while s > 0:
        o = chr(s & 0xFF) + o
        s = s >> 8
    return o


key = "abcd1234ABCD!@#$zyxwZYXW*1*2*3*4"
assert len(key) == 32
iv =  "54123892jsdkjsdj"
assert len(iv) == 16


In [149]:
string1 = "helloworld123456"
string2 = "test string 1234"

a = AES(packtext(key))

e1 = hex(a.encrypt(packtext(string1)))
#assert e1 == "0x1708271a0a18bb2e15bd658805297b8d"
e2 = hex(a.encrypt(packtext(string2)))
#assert e2 == "0x482ac205196a804865262a0044915738"
print(e1)
print(e2)

print(packtext(key), packtext(string1), int(e1, 0))

a = AES(packtext(key))
print(unpacktext(a.decrypt(int(e1, 0))))
#assert(unpacktext(a.decrypt(int(e1, 0))) == string1)
print(unpacktext(a.decrypt(int(e2, 0))))
#assert(unpacktext(a.decrypt(int(e2, 0))) == string2)



0x2d1541c695f88a16f8bfb5dbe3a95022
0xaad2f0d199eba53714f739b449c90ca6
44048183298989073697966048947243823604726133612511607082607857744689369393716 138766332635719238849554048983485396278 59925632134564593020758952134513872930
helloworld123456
test string 1234


In [150]:
hex(30614575354952859734368363414031006605)

'0x1708271a0a18bb2e15bd658805297b8d'

In [151]:
a = AES(packtext(key), packtext(iv))

e1 = hex(a.encrypt(packtext(string1)))
#assert e1 == "0x6cbaa5d41d87fc1cb2cde5f49c592554"
e2 = hex(a.encrypt(packtext(string2)))
#assert e2 == "0xb2b95376972f97140a84deda840144a2"
print(e1)
print(e2)

a = AES(packtext(key), packtext(iv))
dec1 = (unpacktext(a.decrypt(int(e1, 0))))
#assert(dec1 == string1)
print(dec1)
dec2 = (unpacktext(a.decrypt(int(e2, 0))))
#assert(dec2 == string2)
print(dec2)



0x8ea7ae770057e1522436c05b3d7448b2
0x683ec16c5385a750b7ecf9297f7b2e7e
helloworld123456
test string 1234


In [152]:
from Crypto.Cipher import AES as AE

In [153]:

cipher = AE.new(key.encode(), AE.MODE_ECB)
ciphertext = cipher.encrypt(string1 + string2)
print(ciphertext.hex()[:32])
print(ciphertext.hex()[32:])
plaintext = cipher.decrypt(ciphertext)
print(plaintext)

2d1541c695f88a16f8bfb5dbe3a95022
aad2f0d199eba53714f739b449c90ca6
b'helloworld123456test string 1234'


In [154]:
cipher = AE.new(key.encode(), AE.MODE_CBC, iv)
ciphertext = cipher.encrypt(string1+string2)
print(ciphertext.hex()[:32])
print(ciphertext.hex()[32:])

cipher = AE.new(key.encode(), AE.MODE_CBC, iv)
plaintext = cipher.decrypt(ciphertext)
print(plaintext)

8ea7ae770057e1522436c05b3d7448b2
683ec16c5385a750b7ecf9297f7b2e7e
b'helloworld123456test string 1234'
